In [2]:
import pandas as pd
import datetime
from datetime import timedelta
from datetime import date
import openpyxl
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font
from pathlib import Path
import sys
import os
import time
from openpyxl.styles import Font, NamedStyle, Alignment

start=time.time()

df=pd.read_excel(f'Выгрузка.xlsx')

df['Время модиф-ии'] = pd.to_datetime(df['Время модиф-ии'], format ="%Y.%m.%d %H:%M:%S")
dd = (datetime.datetime.today()-timedelta(30)).strftime("%Y-%m-%d %H:%M:%S")

df["Имя сервиса"]=np.where((df["Оценка сервиса"]<=3), df["Имя сервиса"], np.nan) 
df=df[df["Имя сервиса"].notnull()]

df["Имя сервиса"]=np.where((df["Ответ к оценке"]=="-"), df["Имя сервиса"], np.nan) 
df=df[df["Имя сервиса"].notnull()]

df["Имя сервиса"]=np.where((df["Время модиф-ии"]>=dd), df["Имя сервиса"], np.nan) 
df=df[df["Имя сервиса"].notnull()]

df["Имя сервиса"]=np.where((df["Комментарий оценки"]!="-"), df["Имя сервиса"], np.nan) 
df=df[df["Имя сервиса"].notnull()]

df["Имя сервиса"]=np.where(((df["Тип сервиса"]!="СберДруг")&(df["Тип сервиса"]!="#н/д#")), df["Имя сервиса"], np.nan) 
df=df[df["Имя сервиса"].notnull()]

df['ID Сервиса'] = df['ID Сервиса'].astype(str)


del df["ID Оценщика"]
del df["ФИО оценщика"]
del df["В расчет рейтинга"]
del df["Число лайков оценки"]
del df["Дата-время ответа"]
del df["Ответивший сотрудник"]
del df["1й орг.стр"]
del df["2й орг.стр"]
del df["1й терр"]
del df["2й терр"]
del df["Организация"]
del df["Оргструктура"]
del df["Ответ к оценке"]
del df["Блок"]
del df["Рабочий e-mail"]
del df["Внешний e-mail"]
del df["Сберджайл команда"]
del df["Сберджайл трайб"]
del df["Область экспертизы"]

df.to_excel(f'Итог.xlsx', index=False)


df1=pd.read_excel(f'Спр ТБ.xlsx')
df1.to_excel(f'Спр ТБ.xlsx', index=False)


df2=pd.read_excel(f'Мип.xlsx')
df2['GUID_МИП'] = df2['GUID_МИП'].astype(str)
df2.to_excel(f'Мип.xlsx', index=False)


df=pd.read_excel(f'Справочник по МПУ.xlsx')
lst_a = df['Имя сервиса'].tolist()
df = df.set_index('Имя сервиса')

df2=pd.read_excel(f'Мип.xlsx')
lst_a2 = df2['GUID_МИП'].tolist()
df2 = df2.set_index('GUID_МИП')

df1=pd.read_excel(f'Спр ТБ.xlsx')
lst_a1 = df1['ID Сервиса'].tolist()
df1 = df1.set_index('ID Сервиса')

wb=openpyxl.load_workbook(f'Итог.xlsx')
ws=wb.active
rows = ws.max_row

ws.cell(row=1, column=7).value="Контрольный срок"
ws.cell(row=1, column=8).value="Ссылка на сервис"
ws.cell(row=1, column=9).value="МПУ"
ws.cell(row=1, column=10).value="Направление"
ws.cell(row=1, column=11).value="ТБ"

for i in range(2, rows+1):
    ws.cell(row=i, column=7).value=(ws.cell(row=i, column=5).value+timedelta(5))
    if ws.cell(row=i, column=2).value in lst_a:
        ws.cell(row=i, column=9).value=df.loc[ws.cell(row=i, column=2).value, "Менеджер предоставления услуги"]
        ws.cell(row=i, column=10).value=df.loc[ws.cell(row=i, column=2).value, "Направление"]

for i in range(2, rows+1):        
    if ws.cell(row=i, column=1).value=="ЕСО Шаблон" or ws.cell(row=i, column=1).value=="SMIT Шаблон":
        ws.cell(row=i, column=8).value="https://sberfriend.ca.sbrf.ru/sberfriend/#/interaction/new?elementId="+ ws.cell(row=i, column=3).value
    elif ws.cell(row=i, column=1).value=="МиП":
        ws.cell(row=i, column=8).value="https://sberfriend.ca.sbrf.ru/sberfriend/#/application/" +  ws.cell(row=i, column=3).value

for i in range(2, rows+1):
    if ws.cell(row=i, column=3).value in lst_a1:
        ws.cell(row=i, column=11).value = df1.loc[ws.cell(row=i, column=3).value, "ТБ"]
        
        if ws.cell(row=i, column=11).value == "МИП-ссылка":
            ws.cell(row=i, column=1).value =""
            ws.cell(row=i, column=2).value =""
            ws.cell(row=i, column=3).value =""
            ws.cell(row=i, column=4).value =""
            ws.cell(row=i, column=5).value =""
            ws.cell(row=i, column=6).value =""
            ws.cell(row=i, column=7).value =""
            ws.cell(row=i, column=8).value =""
            ws.cell(row=i, column=9).value =""
            ws.cell(row=i, column=10).value =""
            ws.cell(row=i, column=11).value =""

for i in range(2, rows+1):
    if ws.cell(row=i, column=3).value in lst_a2:
        ws.cell(row=i, column=10).value = df2.loc[ws.cell(row=i, column=3).value, "НАПРАВЛЕНИЕ"]    
        ws.cell(row=i, column=9).value = df2.loc[ws.cell(row=i, column=3).value, "ФИО владельца сервиса"]    

for i in range(2, rows+1):
    if ws.cell(row=i, column=6).value==None and ws.cell(row=i, column=1).value!="":
        ws.cell(row=i, column=6).value="-"
    else: continue
        
wb.save(f'Итог.xlsx')

df=pd.read_excel(f'Итог.xlsx')
df=df.sort_values("Тип сервиса")
del df["ID Сервиса"] 
df=df[["ТБ", "Направление", "МПУ", "Тип сервиса", "Имя сервиса", "Оценка сервиса", "Комментарий оценки", "Время модиф-ии", "Контрольный срок", "Ссылка на сервис"]]

df["Тип сервиса"]=np.where((df["Комментарий оценки"]!="-"), df["Тип сервиса"], np.nan) 
df=df[df["Тип сервиса"].notnull()]

df.to_excel(f'Итог.xlsx', index=False)

wb=openpyxl.load_workbook(f'Итог.xlsx')
ws=wb.active
rows = ws.max_row

ws.column_dimensions['A'].width = 9
ws.column_dimensions['B'].width = 30
ws.column_dimensions['C'].width = 30
ws.column_dimensions['D'].width = 16
ws.column_dimensions['E'].width = 35
ws.column_dimensions['F'].width = 17
ws.column_dimensions['G'].width = 25
ws.column_dimensions['H'].width = 20
ws.column_dimensions['I'].width = 20
ws.column_dimensions['J'].width = 36

for i in range(1, 600):
    for j in range(1, 600):
        ws.cell(row=i, column=j).font= Font(size=12, name ='Calibri')
        
def business_days(since, until):
    since_isoweekday = since.isoweekday() + 1
    return len([x for x in range(since_isoweekday,
                                 since_isoweekday + (until - since).days)
                if x % 7 not in [0, 6]]) 


for i in range(2, rows+1):
    since = ws.cell(row=i, column=8).value
    until = ws.cell(row=i, column=9).value
    ws.cell(row=i, column=11).value=business_days(since, until)
    if ws.cell(row=i, column=11).value==3:
        ws.cell(row=i, column=9).value=ws.cell(row=i, column=8).value+timedelta(7)
    elif ws.cell(row=i, column=11).value==4:
        ws.cell(row=i, column=9).value=ws.cell(row=i, column=8).value+timedelta(6)
    daysw=ws.cell(row=i, column=9).value
    ws.cell(row=i, column=12).value=daysw.weekday()
    if ws.cell(row=i, column=12).value==6:
        ws.cell(row=i, column=9).value=ws.cell(row=i, column=8).value+timedelta(7)
        
for i in range(2, rows+1): 
    ws.cell(row=i, column=11).value=""
    ws.cell(row=i, column=12).value=""
    
wb.save(f'Итог.xlsx')


print("Отчет готов")
end=time.time()
print("Время выполнения программы: ", end-start, "сек")

Отчет готов
Время выполнения программы:  78.82328152656555 сек
